In [ ]:
# Created by: Brandon Susini
# Date: 2/14/2023

# Homework 3

### Assemble and clean data from all 5 groups

First, we import the necessary libraries
<ul>
<li>csv allows us to work with csv file types easily.</li>
<li>pandas gives us access to many data science related tools like data frames</li>

In [16]:
import csv, pandas as pd
# Make a list of all the expected column names for the documents
column_names = ['Url', 'Date', 'Content', 'Id', 'User', 'Reply Count',
       'Retweet Count', 'Like Count', 'Quote Count', 'Conversation Id',
       'Language', 'Source', 'Coordinates']
# Convert all the names to lowercase to make it easier in the future.
for index, name in enumerate(column_names):
       column_names[index] = name.lower()

Load each csv file into a dataframe.<br>
<strong>Note:</strong>
Some groups have bugged column names and other issues.<br>
For this reason we have to validate the order and value of the 
column names.

In [17]:
# List to hold each groups documents
all_docs = []
# Iterate 5 times in order load in all the groups csv files
for index in range(1,6):
    # Build the path for the current document
    current_file = f"../all_dirty_docs/tweets_{index}.csv"
    # Load and save the document into a variable
    this_document = pd.read_csv(filepath_or_buffer=current_file)
    # Create a dictionary that has
    # The column name as a key, and the lowered column name as a value
    column_dict = {}
    for name in this_document.columns.values:
        column_dict[name] = name.lower()
    # Rename all the columns to their lowered version
    this_document.rename(columns=column_dict,inplace=True)
    # Add the document to the tweets_list list.
    all_docs.append(this_document)
    # First, check to see if the number of columns matches.
    print(len(this_document.columns.values), " vs. ",len(column_names))
    


14  vs.  13
14  vs.  13
12  vs.  13
14  vs.  13
14  vs.  13


In [18]:
#%%script echo skipping
for x in range(0,5):
    print(all_docs[x].columns)

Index(['unnamed: 0', 'url', 'date', 'content', 'id', 'user', 'reply count',
       'retweet count', 'like count', 'quote count', 'conversation id',
       'language', 'source', 'coordinates'],
      dtype='object')
Index(['url', 'date', 'content', 'id', 'user', 'reply count', 'retweet count',
       'like count', 'quote count', 'conversation id', 'language', 'source',
       'coordinates', 'unnamed: 13'],
      dtype='object')
Index(['date', 'user', 'content', 'like count', 'id', 'reply count',
       'retweet count', 'quote count', 'conversation id', 'language', 'source',
       'coordinates'],
      dtype='object')
Index(['unnamed: 0', 'url', 'date', 'content', 'user', 'reply count',
       'retweet count', 'like count', 'quote count', 'id', 'language',
       'source', 'coordinates', 'dashes'],
      dtype='object')
Index(['unnamed: 0', 'url', 'date', 'content', 'id', 'user', 'reply count',
       'retweet count', 'like count', 'quote count', 'conversation id',
       'language', 's

This clearly shows that all the documents need to be fixed.

Because our data is inconsistent, we need to re-organize it.
We start by determining if the group removed the label headers for each of the 200 tweets.
If a document doesn't have a max ID of ~1200 then we can assume the ID's reset at 199.

Furthermore, we must ensure that each document has the same order of the columns so we can join them cleanly.

In [19]:
# Figure out what the maximum "ID" is for a frame.
# Before we can do this we need to determine what column is the 
# bugged "ID" column.
# In the case of tweets_1, it is the column "unnamed: 0"
# Therefore, any document with a column named "unnamed: 0"
# will need to undergo this process

# REFACTOR THIS SO I CAN CALL IT AS A FUNCTION AND IT WILL TEST
# ALL DOCUMENTS WITHIN 
header_docs = [all_docs[0],all_docs[3],all_docs[4]]
my_max = 0
for doc in header_docs:
    print(f"Num of rows before: {doc.shape[0]}")
    # Removes all instances of header columns by removing any row where "Unnamed: 0" == NaN
    doc.dropna(subset=["unnamed: 0"], inplace=True)
    # Now remove the column all together and re-index the data frame.
    doc.drop(labels="unnamed: 0",axis=1,inplace=True)
    doc.reset_index(inplace=True,drop=True)
    # Finally, drop the created "index" column.
    print(f"Num of rows after: {doc.shape[0]}")
all_docs[1].drop(labels="unnamed: 13",axis=1,inplace=True)
    

# Following this the documents 1 and 5 should be fixed and correctly indexed
# and stripped of the problem columns
#
# We still need to fix document 4 however.

# Same thing - Reminder
# print(type(tweets_list[0].Url))
# print(type(tweets_list[0]["Url"]))

Num of rows before: 1208
Num of rows after: 1200
Num of rows before: 1031
Num of rows after: 1031
Num of rows before: 1208
Num of rows after: 1200


In [20]:
for x in range(0,5):
    print(all_docs[x].columns)

Index(['url', 'date', 'content', 'id', 'user', 'reply count', 'retweet count',
       'like count', 'quote count', 'conversation id', 'language', 'source',
       'coordinates'],
      dtype='object')
Index(['url', 'date', 'content', 'id', 'user', 'reply count', 'retweet count',
       'like count', 'quote count', 'conversation id', 'language', 'source',
       'coordinates'],
      dtype='object')
Index(['date', 'user', 'content', 'like count', 'id', 'reply count',
       'retweet count', 'quote count', 'conversation id', 'language', 'source',
       'coordinates'],
      dtype='object')
Index(['url', 'date', 'content', 'user', 'reply count', 'retweet count',
       'like count', 'quote count', 'id', 'language', 'source', 'coordinates',
       'dashes'],
      dtype='object')
Index(['url', 'date', 'content', 'id', 'user', 'reply count', 'retweet count',
       'like count', 'quote count', 'conversation id', 'language', 'source',
       'coordinates'],
      dtype='object')


### Left off after finishing the max calculation loop for problem frames
When you come back re-read that section and figure out how to turn it into a function that
will operate off of tweets_list

Once you finish that you will have a repeateable function that will make sure indecides are correct

Then you can start working on fixing column names for all the dataframes.

In [21]:
#%%script echo skipping
# Now that those problem documents are fixed we need to make all the tables
# Have the same column names and order of names.
# Then find problem columns by checking the column names
# against our expected column names.
for index, doc in enumerate(all_docs):
    rename_list = []
    # For dataframe.rename we need a dict with key = current name
    # value = desired name
    for name in column_names:
        if name not in doc.columns.values:
            print(f"Document {index+1} missing column \"{name}\"")
            

        

Document 3 missing column "url"
Document 4 missing column "conversation id"


In [22]:
# With the previous cell we make sure all column names are present.
# Now we need to reorder them
corpus = pd.concat(all_docs)
corpus.drop(labels="dashes",axis=1,inplace=True)
display(corpus)



,url,date,content,id,user,reply count,retweet count,like count,quote count,conversation id,language,source,coordinates
0,https://twitter.com/strombone1/status/16202252...,2023-01-31 00:59:29+00:00,https://t.co/ZGWUKFL10w,1.62E+18,https://twitter.com/strombone1,70,21,498,6,1.62E+18,zxx,"<a href=""http://twitter.com/download/iphone"" r...",NaN
1,https://twitter.com/strombone1/status/16202252...,2023-01-31 00:59:27+00:00,I’m a man of the people. Which helmet should I...,1.62E+18,https://twitter.com/strombone1,153,51,429,20,1.62E+18,en,"<a href=""http://twitter.com/download/iphone"" r...",NaN
2,https://twitter.com/strombone1/status/16201901...,2023-01-30 22:39:57+00:00,@JessBlaylock @emilymkaplan First line center ...,1.62E+18,https://twitter.com/strombone1,15,10,412,1,1.62E+18,en,"<a href=""http://twitter.com/download/iphone"" r...",NaN
3,https://twitter.com/strombone1/status/16056389...,2022-12-21 18:59:02+00:00,And people think Santa doesn’t exist https://t...,1.61E+18,https://twitter.com/strombone1,177,294,6478,35,1.61E+18,en,"<a href=""http://twitter.com/download/iphone"" r...",NaN
4,https://twitter.com/strombone1/status/15984627...,2022-12-01 23:43:20+00:00,Finally got my captains photo done https://t.c...,1.60E+18,https://twitter.com/strombone1,155,504,9829,44,1.60E+18,en,"<a href=""http://twitter.com/download/iphone"" r...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,https://twitter.com/RafaelNadal/status/1414319...,2021-07-11 20:24:07+00:00,Congrats @DjokerNole on this amazing achieveme...,1414319614474035203,https://twitter.com/RafaelNadal,737,6723,79425,790,1414319614474035203,en,"<a href=""http://twitter.com/download/iphone"" r...",NaN
1196,https://twitter.com/RafaelNadal/status/1413228...,2021-07-08 20:10:12+00:00,"Hi, everyone! Join me on @FanJoltLive where we...",1413228947802923014,https://twitter.com/RafaelNadal,64,116,1227,11,1413228947802923014,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN
1197,https://twitter.com/RafaelNadal/status/1409610...,2021-06-28 20:30:55+00:00,👏🏻👏🏻👏🏻👏🏻👏🏻👏🏻👏🏻👏🏻,1409610285107789834,https://twitter.com/RafaelNadal,67,176,2879,12,1409610285107789834,art,"<a href=""http://twitter.com/download/iphone"" r...",NaN
1198,https://twitter.com/RafaelNadal/status/1409610...,2021-06-28 20:30:17+00:00,Partidazo de España hoy! 🇪🇸Enhorabuena chicos ...,1409610125724237836,https://twitter.com/RafaelNadal,99,694,11269,41,1409610125724237836,es,"<a href=""http://twitter.com/download/iphone"" r...",NaN


In [23]:
%%script echo skipping
corpus.to_csv(path_or_buf="all_tweets_group_4.csv",index=False)